# Windstorm tracks and footprints derived from reanalysis over Europe between 1940 to present: Windstorm statistics

Production date: YYYY-MM-DD

**Please note that this repository is used for development and review, so quality assessments should be considered work in progress until they are merged into the main branch.**

Produced by: Olivier Burggraaff (National Physical Laboratory).

## 🌍 Use case: Use case listed here in full 

## ❓ Quality assessment question
* **In most cases there should be one question listed here in bold**

**‘Context paragraph’ (no title/heading)** - a very short introduction before the assessment statement describing approach taken to answer the user question. One or two key references could be useful, if the assessment summarises literature.

**Background**

## 📢 Quality assessment statement

```{admonition} These are the key outcomes of this assessment
:class: note
* Finding 1
* Finding 2
* Finding 3
* etc
```

## 📋 Methodology

* Internal consistency: TempestExtremes vs TRACK/Hodges? (read papers)
* 

A ‘free text’ introduction to the data analysis steps or a description of the literature synthesis, with a justification of the approach taken, and limitations mentioned. **Mention which CDS catalogue entry is used, including a link, and also any other entries used for the assessment**.

[Windstorm tracks and footprints derived from reanalysis over Europe between 1940 to present
](https://doi.org/10.24381/bf1f06a9)

Hodges (TRACK) algorithm [[Hoskins+02](https://doi.org/10.1175/1520-0469(2002)059%3C1041:NPOTNH%3E2.0.CO;2), [Hodges+99](https://doi.org/10.1175/1520-0493(1999)127%3C1362:ACFFT%3E2.0.CO;2), [Hodges+95](https://doi.org/10.1175/1520-0493(1995)123%3C3458:FTOTUS%3E2.0.CO;2)]

TempestExtremes [[Ullrich+21](https://doi.org/10.5194/gmd-14-5023-2021), [Ullrich+17](https://doi.org/10.5194/gmd-10-1069-2017)]

**Note:** This notebook is currently just a brain-dump in anticipation of starting the actual quality assessment at a later stage.

E.g. 'The analysis and results are organised in the following steps, which are detailed in the sections below:' 

**[](section-setup)**
 * Sub-steps or key points listed in bullet below. No strict requirement to match and link to sub-headings.

**[](section-download)**
 * Sub-steps or key points listed in bullet below. No strict requirement to match and link to sub-headings.

**[](section-analysis)**
 * Sub-steps or key points listed in bullet below. No strict requirement to match and link to sub-headings.

**[](section-results)**
 * Sub-steps or key points listed in bullet below. No strict requirement to match and link to sub-headings.

Any further notes on the method could go here (explanations, caveats or limitations).

## 📈 Analysis and results

(section-setup)=
### 1. Code setup
```{note}
This notebook uses [earthkit](https://github.com/ecmwf/earthkit) for
downloading ([earthkit-data](https://github.com/ecmwf/earthkit-data))
and visualising ([earthkit-plots](https://github.com/ecmwf/earthkit-plots)) data.
Because earthkit is in active development, some functionality may change after this notebook is published.
If any part of the code stops functioning, please raise an issue on our GitHub repository so it can be fixed.
```

#### Imports

In [ ]:
# Input / Output
from pathlib import Path
import earthkit.data as ekd
import warnings

# General data handling
import numpy as np
np.seterr(divide="ignore")  # Ignore divide-by-zero warnings
import pandas as pd
import geopandas as gpd
import xarray as xr
from functools import partial
from dask.array.core import PerformanceWarning
warnings.simplefilter(action="ignore", category=PerformanceWarning)

# Visualisation
import earthkit.plots as ekp
from earthkit.plots.styles import Style
import matplotlib.pyplot as plt
plt.rcParams["grid.linestyle"] = "--"
from matplotlib.colors import LogNorm
from cartopy import crs as ccrs
import cmcrameri as cmc
from tqdm import tqdm  # Progress bars

# Visualisation in Jupyter book -- automatically ignored otherwise
try:
    from myst_nb import glue
except ImportError:
    glue = None

#### Helper functions

##### General

In [ ]:
# Type hints for helper functions
from typing import Callable, Optional, Iterable

# For pre-defining functions
from functools import partial

Constants:

In [ ]:
TRACKING_ALGORITHMS = ["hodges", "tempest_extremes"]  # Can be re-used later for comparisons

##### Downloading data

In [ ]:
## Data downloading
def domain_to_request(domain: ekp.geo.domains.Domain) -> dict:
    """ From an earthkit-plots domain, generate a request for earthkit-data / cdsapi. """
    bbox = domain.bbox.to_latlon_bbox()

    # Round
    north = int(np.ceil(bbox.north) + 1)
    south = int(np.floor(bbox.south) - 1)
    west = int(np.floor(bbox.west) - 1)
    east = int(np.ceil(bbox.east) + 1)
    
    area = [north, west, south, east]
    return {"area": area}

##### Data (pre-)processing

In [ ]:
## Loops for convenience
def loop_over_(*args, progress=True, **kwargs) -> tqdm:
    """ Generate a tqdm progressbar; inverts `disable` keyword """
    return tqdm(*args, disable=not progress, leave=False, **kwargs)

def loop_over_ensemble_members(data: xr.Dataset, **tqdm_kwargs) -> tqdm:
    """ Loop over ensemble members in `data`, with a progress bar. """
    return loop_over_(data.groupby("number"), unit="member", **tqdm_kwargs)

def loop_over_data_variables(data: xr.Dataset, **tqdm_kwargs) -> tqdm:
    """ Loop over variable keys in `data`, with a progress bar. """
    return loop_over_(data.data_vars.keys(), unit="variable", **tqdm_kwargs)

##### Visualisation

In [ ]:
_style_footprint = {"cmap": plt.cm.cividis, "vmin": 0, "vmax": 40}

styles = {
    "footprint": Style(**_style_footprint),
}

# Apply general settings
for style in styles.values():
    style.normalize = False

In [ ]:
def _add_textbox_to_subplots(text: str, *axs: Iterable[plt.Axes | ekp.Subplot], right=False) -> None:
    """ Add a text box to each of the specified subplots. """
    # Get the plt.Axes for each ekp.Subplot
    axs = [subplot.ax if isinstance(subplot, ekp.Subplot) else subplot for subplot in axs]

    # Set up location
    x = 0.95 if right else 0.05
    horizontalalignment = "right" if right else "left"

    # Add the text
    for ax in axs:
        ax.text(x, 0.95, text, transform=ax.transAxes,
        horizontalalignment=horizontalalignment, verticalalignment="top",
        bbox={"facecolor": "white", "edgecolor": "black", "boxstyle": "round",
              "alpha": 1})

In [ ]:
def decorate_fig(fig: ekp.Figure, *, title: Optional[str]="") -> None:
    """ Decorate an earthkit figure with land, coastlines, etc. """
    # Add progress bar because individual steps can be very slow for large plots
    with tqdm(total=4, desc="Decorating", leave=False) as progressbar:
        fig.land()
        progressbar.update()
        fig.coastlines()
        progressbar.update()
        # fig.borders()
        # progressbar.update()
        fig.gridlines(linestyle=plt.rcParams["grid.linestyle"])
        progressbar.update()
        fig.title(title)
        progressbar.update()

(section-download)=
### 2. Download data

#### CDS: Windstorm summary indicators
We download the full windstorm summary indicator table from the
[sis-european-wind-storm-reanalysis](https://doi.org/10.24381/bf1f06a9)
CDS entry.
Because this is a simple table,
the data are loaded into a Pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html).
Geographical information will be added to this table in the next subsection.

In [ ]:
dataset = "sis-european-wind-storm-reanalysis"

In [ ]:
request = {
    "product": "windstorm_summary_indicator",
    "variable": "all",
    "tracking_algorithm": ["hodges", "tempestextremes"],
    # Note that we do not use the TRACKING_ALGORITHMS constant here because the CDS
    # does not have an _ in `tempestextremes` but the resulting table does.
}

In [ ]:
data = ekd.from_source("cds", dataset, request)
data = data.to_pandas()
data

#### Ancillary: NUTS shapefiles
The summary table is organised by NUTS
(Nomenclature of Territorial Units for Statistics)
level 0, 1, and 2
region.
To visualise these areas,
we need the corresponding shapefiles,
which can be retrieved from [GISCO](https://ec.europa.eu/eurostat/web/gisco/geodata/statistical-units/territorial-units-statistics).
We load these data into a GeoPandas [GeoDataFrame](https://geopandas.org/en/latest/docs/reference/api/geopandas.GeoDataFrame.html):

In [ ]:
# Set up GISCO URL and request
base = "https://gisco-services.ec.europa.eu/distribution/v2/nuts/geojson/"
fname = "NUTS_RG_20M_2024_3035_LEVL_{level}.geojson"
# RG: Polygons
# 20M: 1:20 Million scale
# 2024: Year
# 3035: EPSG CRS (projection)
# LEVEL: Iterate over 0, 1, 2
urls = [base + "/" + fname.format(level=level) for level in (0, 1, 2)]

# Download data from GISCO
nuts_gdf = [gpd.read_file(url) for url in urls]

# Merge NUTS levels into one table
nuts_gdf = gpd.GeoDataFrame(pd.concat(nuts_gdf))
nuts_gdf = nuts_gdf.rename(columns={"NUTS_ID": "region"})

The NUTS shapes are merged into the CDS data:

In [ ]:
# Merge NUTS polygons into data
data_nuts = nuts_gdf.merge(data, on="region")

# Filter only desired columns
data_nuts = data_nuts[["region", "year", "threshold", "algorithm",  # Selection
                       "storms_number", "mean_wind_gust", "ssi", "normalised_ssi", "area_ratio", "wind_gust_ratio",  # Variables
                       "LEVL_CODE", "CNTR_CODE", "NAME_LATN", "geometry",  # NUTS
                     ]]

data_nuts

(section-analysis)=
### 3. Analysis
Split into tabular and geospatial for now because easier to experiment.
Final analysis should probably be by topic rather than by visualisation.

#### Tabular comparison

In [ ]:
#

#### Geospatial comparison
Note: Choropleth maps are not supported in earthkit-plots yet, but will be soon. See GitHub issue [#160](https://github.com/ecmwf/earthkit-plots/issues/160).

In [ ]:
# Nuts 0

selection = (data_nuts["year"] == 2004) & \
            (data_nuts["threshold"] < 1) & \
            (data_nuts["algorithm"] == "hodges") & \
            (data_nuts["LEVL_CODE"] == 0)

data_national = data_nuts[selection]

data_national.plot.geo("storms_number")
plt.show()
plt.close()

In [ ]:
# Nuts 1

In [ ]:
# Nuts 2

In [ ]:
# For earthkit-plots issue #160
# data_nuts[selection].to_file("choropleth_example.geojson", driver='GeoJSON')

(section-results)=
### 5. Results

#### Results Subsections
Describe what is done in this step/section and what the `code` in the cell does (if code is included). 

If this is the **results section**, we expect the final plots to be created here with a description of how to interpret them, and what information can be extracted for the specific use case and user question. The information in the 'quality assessment statement' should be derived here. 

## ℹ️ If you want to know more

### Key resources

Code libraries used:
* [GeoPandas](https://geopandas.org/en/stable/)
* [earthkit](https://github.com/ecmwf/earthkit)
  * [earthkit-data](https://github.com/ecmwf/earthkit-data)
  * [earthkit-plots](https://github.com/ecmwf/earthkit-plots)

### References
[[Hoskins+02](https://doi.org/10.1175/1520-0469(2002)059%3C1041:NPOTNH%3E2.0.CO;2)] B. J. Hoskins and K. I. Hodges, ‘New Perspectives on the Northern Hemisphere Winter Storm Tracks’, Journal of the Atmospheric Sciences, vol. 59, no. 6, pp. 1041–1061, Mar. 2002, doi: 10.1175/1520-0469(2002)059%3C1041:NPOTNH%3E2.0.CO;2.

[[Hodges+99](https://doi.org/10.1175/1520-0493(1999)127%3C1362:ACFFT%3E2.0.CO;2)] K. I. Hodges, ‘Adaptive Constraints for Feature Tracking’, Monthly Weather Review, vol. 127, no. 6, pp. 1362–1373, June 1999, doi: 10.1175/1520-0493(1999)127%3C1362:ACFFT%3E2.0.CO;2.

[[Hodges+95](https://doi.org/10.1175/1520-0493(1995)123%3C3458:FTOTUS%3E2.0.CO;2)] K. I. Hodges, ‘Feature Tracking on the Unit Sphere’, Monthly Weather Review, vol. 123, no. 12, pp. 3458–3465, Dec. 1995, doi: 10.1175/1520-0493(1995)123%3C3458:FTOTUS%3E2.0.CO;2.

[[Ullrich+21](https://doi.org/10.5194/gmd-14-5023-2021)] P. A. Ullrich, C. M. Zarzycki, E. E. McClenny, M. C. Pinheiro, A. M. Stansfield, and K. A. Reed, ‘TempestExtremes v2.1: a community framework for feature detection, tracking, and analysis in large datasets’, Geoscientific Model Development, vol. 14, no. 8, pp. 5023–5048, Aug. 2021, doi: 10.5194/gmd-14-5023-2021.

[[Ullrich+17](https://doi.org/10.5194/gmd-10-1069-2017)] P. A. Ullrich and C. M. Zarzycki, ‘TempestExtremes: a framework for scale-insensitive pointwise feature tracking on unstructured grids’, Geoscientific Model Development, vol. 10, no. 3, pp. 1069–1090, Mar. 2017, doi: 10.5194/gmd-10-1069-2017.